In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
from pandas.core.common import SettingWithCopyWarning
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta, date
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
# Load in price data
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')
occ_data.id = occ_data.id.astype('str')
miami_example = occ_data[occ_data['id'] == '52364652.0']
#start_time = time.time()
#miami_example['pulled'] = pd.to_datetime(miami_example['pulled'])
#print('done with datatype change')
#middle_time = time.time()
#miami_example['pulled'] = miami_example['pulled'].dt.date
#print('done with change to date')
#end_time = time.time()

#print('Start Time '+ str(start_time))
#print('Middle Time '+ str(middle_time-start_time))
#print('End Time '+ str(end_time-start_time))


"\nstart_time = time.time()\nmiami_example['pull_time'] = pd.to_datetime(miami_example['pull_time'])\nprint('done with datatype change')\nmiddle_time = time.time()\nmiami_example['pull_time'] = miami_example['pull_time'].dt.date\nprint('done with change to date')\nend_time = time.time()\n\nprint('Start Time '+ str(start_time))\nprint('Middle Time '+ str(middle_time-start_time))\nprint('End Time '+ str(end_time-start_time))\n\nmiami_example_by_pull_time = miami_example.groupby(['id','pull_time'])['cleaning_fee','service_fee','total_price'].mean().reset_index()\nmiami_example_by_pull_time_count = miami_example.groupby(['pull_time','check_in'])['cleaning_fee','service_fee','total_price'].count().reset_index()\n"

In [6]:
miami_example_by_pull_time = miami_example.groupby(['id','pulled'])['available','min_nights','available_for_checkin','available_for_checkout','bookable'].mean().reset_index()
miami_example_by_pull_time_count = miami_example.groupby(['pulled','date'])['available','min_nights','available_for_checkin','available_for_checkout','bookable'].count().reset_index()

# Create traces
miami_example_grouped_fig = go.Figure()
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available'],
                    mode='lines',
                    name='Available'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available_for_checkin'],
                    mode='lines', 
                    name='Available for Check In'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available_for_checkout'],
                    mode='lines', 
                    name='Available for Check Out'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['bookable'],
                    mode='lines', 
                    name='Bookable'))
miami_example_grouped_fig.update_layout(title_text="Occupancy Data by Date", xaxis_title='Date', yaxis_title='Availability')
filename = "newsletter_features/occupancy_data_line_fig_miami_example.png"
miami_example_grouped_fig.write_image(filename, engine='kaleido', width=2000, height=500)
miami_example_grouped_fig.show()

In [4]:
miami_example

,index,date,available,max_nights,min_nights,available_for_checkin,available_for_checkout,bookable,pulled,id,parition_col
8880085,0,2022-04-01,False,1125,5,False,False,False,18/04/2022 23:52:12,52364652.0,12.0
8880086,1,2022-04-02,False,1125,5,False,False,False,18/04/2022 23:52:12,52364652.0,12.0
8880087,2,2022-04-03,False,1125,5,False,False,False,18/04/2022 23:52:12,52364652.0,12.0
8880088,3,2022-04-04,False,1125,5,False,False,False,18/04/2022 23:52:12,52364652.0,12.0
8880089,4,2022-04-05,False,1125,5,False,False,False,18/04/2022 23:52:12,52364652.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
10678800,360,2023-04-26,False,1125,5,False,False,False,11/05/2022 06:22:37,52364652.0,12.0
10678801,361,2023-04-27,False,1125,5,False,False,False,11/05/2022 06:22:37,52364652.0,12.0
10678802,362,2023-04-28,False,1125,5,False,False,False,11/05/2022 06:22:37,52364652.0,12.0
10678803,363,2023-04-29,False,1125,5,False,False,False,11/05/2022 06:22:37,52364652.0,12.0
